<a href="https://colab.research.google.com/github/amarbtsingh/first-contributions/blob/main/song_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Song playlist and Songs metadata datasets

In [1]:
# Imports
import pandas as pd
from urllib import request

In [15]:
# Get the playlist dataset file
url = 'https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

# Create a Request object with headers
req = request.Request(url, headers=headers)
data = request.urlopen(req)

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

In [17]:
# Load song metadata
url = 'https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

req = request.Request(url, headers=headers)
songs_file = request.urlopen(req)

# songs_file = request.urlopen('https://storage.googleapis.com/mapspremium/\
# dataset/yes_complete/song_hash.txt')

songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title','artist'])
songs_df = songs_df.set_index('id')

In [18]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

# Lets Train the Model

In [20]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 69.1 MB/s eta 0:00:00


In [21]:
from gensim.models import Word2Vec
# Train our Word2Vec model
model = Word2Vec(
playlists, vector_size=32, window=20, negative=50,
min_count=1, workers=4
)

In [23]:
# Find most similar songs given a song id

song_id = 2172
model.wv.most_similar(positive=str(song_id))

[('5586', 0.9983382225036621),
 ('2014', 0.997999370098114),
 ('6685', 0.9973744750022888),
 ('11473', 0.9969488978385925),
 ('3119', 0.9958708882331848),
 ('1922', 0.9958620071411133),
 ('11517', 0.9957305192947388),
 ('6658', 0.9954414367675781),
 ('2104', 0.9953933954238892),
 ('5634', 0.9949024319648743)]

In [26]:
# Song is

print(songs_df.iloc[song_id])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


# Get recommendations

In [28]:
import numpy as np
def print_recommendations(song_id):
  similar_songs = np.array(
  model.wv.most_similar(positive=str(song_id),topn=5))[:,0]
  return songs_df.iloc[similar_songs]

# Extract recommendations
print_recommendations(2172)

,title,artist
id,,
5586,The Last In Line,Dio
2014,Youth Gone Wild,Skid Row
6685,The Trooper,Iron Maiden
11473,Little Guitars,Van Halen
3119,There's Only One Way To Rock,Sammy Hagar
